<a href="https://polly.elucidata.io/manage/workspaces?action=open_polly_notebook&amp;source=github&amp;path=ElucidataInc%2Fpolly-python%2Fblob%2Fmain%2FDiscover%2Fh5ad_querying_for_single_cell_data.ipynb&amp;kernel=elucidata/Single-cell Downstream 2.0 &;machine=gp" target="_parent"><img alt="Open in Polly" src="https://elucidatainc.github.io/PublicAssets/open_polly.svg"/></a>


# Welcome to polly python single cell querying Demo 

Flow:-
    1. Summary of the Atlas where data is stored
    2. Schema of different tables in the Atlas
    3. Querying at dataset, var (feature) and obs (sample) tables to understand the format in which data is stored
    4. Advanced queries to slice/dice the data and JOIN/UNION tables as per user's requirement

In [103]:
# import image module
from IPython.display import Image
  
# get the image
Image(url="https://github.com/ElucidataInc/polly-python/blob/main/Discover/single-cell.png?raw=true", width=500, height=300)

In [ ]:
# Install polly python
!sudo pip3 install polly-python

In [2]:
# Import required python modules
import sys
import pandas as pd
from polly.omixatlas import OmixAtlas
from polly.auth import Polly

In [3]:
Polly.auth(os.environ['POLLY_REFRESH_TOKEN'])
library_client = OmixAtlas()
pd.set_option('display.max_columns', 500)

### Summary of Single Cell OmixAtlas

In [23]:
library_client.omixatlas_summary("sc_data_lake")

{'data': {'repo_name': 'sc_data_lake',
  'repo_id': '17',
  'indexes': {'gct_metadata': 'sc_data_lake_gct_metadata',
   'h5ad_metadata': 'sc_data_lake_h5ad_metadata',
   'csv': 'sc_data_lake_csv',
   'files': 'sc_data_lake_files',
   'ipynb': 'sc_data_lake_ipynb',
   'gct_data': 'sc_data_lake_gct_data',
   'h5ad_data': 'sc_data_lake_h5ad_data'},
  'v2_indexes': {'csv': 'sc_data_lake_csv',
   'files': 'sc_data_lake_files',
   'gct_col_metadata': 'sc_data_lake_gct_col_metadata',
   'gct_row_metadata': 'sc_data_lake_gct_row_metadata',
   'h5ad_col_metadata': 'sc_data_lake_h5ad_col_metadata',
   'h5ad_row_metadata': 'sc_data_lake_h5ad_row_metadata',
   'ipynb': 'sc_data_lake_ipynb',
   'json': 'sc_data_lake_json'},
  'sources': [{'geo': 3462},
   {'scp': 264},
   {'expression atlas': 199},
   {'humancellatlas': 7},
   {'publication': 3},
   {'zenodo': 1}],
  'datatypes': [{'single cell': 3939}],
  'dataset_count': 3940,
  'disease_count': 894,
  'tissue_count': 561,
  'organism_count': 17,

### Tables in the Single Cell OmixAtlas

In [22]:
tables = library_client.query_metadata("""SHOW TABLES IN sc_data_lake""")
tables

Query execution succeeded (time taken: 1.50 seconds, data scanned: 0.000 MB)
Fetched 3 rows


,table_name
0,sc_data_lake.datasets
1,sc_data_lake.samples_singlecell
2,sc_data_lake.features_singlecell


### Identifying the datasets available in the Atlas which meets the requirement - Parkinson Disease with more than 20 samples and 1000 cells

In [97]:
relevant_data = library_client.query_metadata("""SELECT dataset_id, curated_disease, curated_tissue, total_num_samples, total_num_cells, data_table_name from sc_data_lake.datasets WHERE 
                                         total_num_samples >20 AND total_num_cells >1000 AND CONTAINS(curated_disease, 'Parkinson Disease')""")
relevant_data

Query execution succeeded (time taken: 1.40 seconds, data scanned: 0.081 MB)
Fetched 4 rows


,dataset_id,curated_disease,curated_tissue,total_num_samples,total_num_cells,data_table_name
0,GSE141578_GPL24676,"[Lewy Body Disease, Normal, Parkinson Disease]","[immune system, cerebrospinal fluid, lewy bo...",22,25294,sc_data_lake__gse141578_gpl24676
1,GSE116138_GPL13112,"[Parkinson Disease, Normal]",[midbrain],1414,1414,sc_data_lake__gse116138_gpl13112
2,GSE118412_GPL16791,[Parkinson Disease],[brain],1396,1396,NaN
3,GSE184950_GPL24676,[Parkinson Disease],[substantia nigra],33,260413,NaN


In [98]:
## Note: Querying on the data matrix will work only if the data_table_name has a valid table name in it.

### Checking schema of each table in the Atlas

In [37]:
schema = library_client.get_schema("1662438213122", schema_level = ["datasets", "samples_singlecell","features_singlecell"], return_type = 'dataframe')

### Dataset level schema

In [38]:
schema.datasets

,Source,Datatype,Field Name,Field Description,Field Type,Is Curated,Is Array
0,all,all,abstract,Abstract of the publication assocaited with th...,text,False,False
1,all,all,author,Name of the author who published the dataset,text,False,False
2,all,all,bucket,S3 bucket in which the data file resides,text,False,False
3,all,all,curated_cell_line,Cell lines from which the samples were derived...,text,True,True
4,all,all,curated_cell_type,Types of cell present in the dataset,text,True,True
5,all,all,curated_disease,Disease associated with the dataset,text,True,True
6,all,all,curated_drug,Drugs administered in the samples belonging to...,text,True,True
7,all,all,curated_gene,Gene studied in the dataset,text,True,True
8,all,all,curated_organism,Orgnism from which the samples were derived,text,True,True
9,all,all,curated_strain,Strain of the organism from which samples were...,text,True,True


### Sample (cells) level schema

In [17]:
schema.samples_singlecell

,Source,Datatype,Field Name,Field Description,Field Type,Is Curated,Is Array
0,all,all,age,Age of the organism from whom the sample was t...,text,False,False
1,all,all,characteristics_ch1,characteristics_ch1,text,False,False
2,all,all,clusters,Cell Type Cluster,integer,False,False
3,all,all,curated_cell_line,Cell line from which the sample was derived,text,True,True
4,all,all,curated_cell_ontology_id,Curated Cell Ontology ID,text,True,False
5,all,all,curated_cell_type,kw_cell_type,text,True,False
6,all,all,curated_cell_type_singler,kw_cell_type_singler,text,True,False
7,all,all,curated_disease,Disease associated with the sample,text,True,True
8,all,all,curated_drug,Drug admistered in the sample,text,True,True
9,all,all,curated_gene,Gene of interest in the sample,text,True,True


### Feature (genes) level schema

In [18]:
schema.features_singlecell

,Source,Datatype,Field Name,Field Description,Field Type,Is Curated,Is Array
0,all,all,data_id,Unique ID for this data entity on Polly,text,False,False
1,all,all,dataset_id,ID of the dataset in which the feature has bee...,text,False,False
2,all,all,feature_id,"ID of the feature (gene, protein, metabolite e...",text,False,False
3,all,all,highly_variable,Whether gene is highly variable,text,False,False
4,all,all,id_key,Name of the key that was used for creation of ...,text,False,False
5,all,all,is_current,Whether this is the current version of the dat...,text,False,False
6,all,all,n_cells,Number of cells containing the gene,integer,False,False
7,all,all,name,Name of the data entity (local to its source f...,text,False,False
8,all,all,src_dataset_id,Dataset ID of the file this data entity origin...,text,False,False
9,all,all,src_repo,Name of the repository this data entity origin...,text,False,False


### Dataset level querying for datasets of interet

In [83]:
datasets = library_client.query_metadata(
    f"""
    SELECT *
    FROM sc_data_lake.datasets
    """
)

Query execution succeeded (time taken: 1.73 seconds, data scanned: 1.466 MB)
Fetched 3978 rows


In [84]:
datasets.head()

,curated_organism,total_num_cells,src_uri,total_num_samples,year,description,curated_cell_line,data_table_name,data_table_version,platform,timestamp_,file_type,publication,curated_cell_type,key,summary,src_repo,drug_smiles,package,file_location,author,dataset_id,curated_disease,curated_drug,curated_gene,abstract,version,curated_strain,bucket,curated_tissue,dataset_source,data_type,overall_design,is_current,region
0,[Mus musculus],4854.0,polly:data://sc_data_lake/data/GSE137710_GPL24...,2.0,2019.0,Transcriptional basis of mouse and human dendr...,[None],NaN,NaN,sc-RNASeq,1662445097233,h5ad,https://pubmed.ncbi.nlm.nih.gov/31668803/,"[dendritic cell, monocyte]",sc_data_lake/data/GSE137710_GPL24247/GSE137710...,Dendritic cells (DCs) play a critical role in ...,sc_data_lake,[],sc_data_lake/data,https://discover-prod-datalake-v1.s3-us-west-2...,NaN,GSE137710_GPL24247,"[Melanoma, Normal, Autoimmune Diseases]",[None],"[CD8A, PTPRC, CD4, TRIM27, CDK1, TBX21]",NaN,0,[],discover-prod-datalake-v1,[spleen],GEO,Single cell,Single-cell RNA sequencing was performed on mo...,True,us-west-2
1,[Homo sapiens],8066.0,polly:data://sc_data_lake/data/GSE137710_GPL24...,3.0,2019.0,Transcriptional basis of mouse and human dendr...,[None],NaN,NaN,sc-RNASeq,1662445416802,h5ad,https://pubmed.ncbi.nlm.nih.gov/31668803/,"[null, Unidentified, myeloid cell, T cell, ...",sc_data_lake/data/GSE137710_GPL24676/GSE137710...,Dendritic cells (DCs) play a critical role in ...,sc_data_lake,[],sc_data_lake/data,https://discover-prod-datalake-v1.s3-us-west-2...,NaN,GSE137710_GPL24676_melanoma,"[Autoimmune Diseases, Melanoma]",[None],"[CD8A, PTPRC, CD4, TRIM27, CDK1, TBX21]",NaN,0,[],discover-prod-datalake-v1,"[skin, spleen]",GEO,Single cell,Single-cell RNA sequencing was performed on mo...,True,us-west-2
2,[Homo sapiens],4668.0,polly:data://sc_data_lake/data/GSE137710_GPL24...,1.0,2019.0,Transcriptional basis of mouse and human dendr...,[None],NaN,NaN,sc-RNASeq,1662445418141,h5ad,https://pubmed.ncbi.nlm.nih.gov/31668803/,"[dendritic cell, Unidentified ]",sc_data_lake/data/GSE137710_GPL24676/GSE137710...,Dendritic cells (DCs) play a critical role in ...,sc_data_lake,[],sc_data_lake/data,https://discover-prod-datalake-v1.s3-us-west-2...,NaN,GSE137710_GPL24676_spleen,"[Autoimmune Diseases, Melanoma]",[None],"[CD8A, PTPRC, CD4, TRIM27, CDK1, TBX21]",NaN,0,[],discover-prod-datalake-v1,"[skin, spleen]",GEO,Single cell,Single-cell RNA sequencing was performed on mo...,True,us-west-2
3,[Homo sapiens],13722.0,polly:data://sc_data_lake/data/GSE137758_GPL18...,3.0,NaN,An OTX2-PAX3 signaling axis regulates Group 3 ...,[None],NaN,NaN,sc-RNASeq,1660730212882,h5ad,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,[],sc_data_lake/data/GSE137758_GPL18573/GSE137758...,scRNA-seq was performed to evaluate the OTX2-P...,sc_data_lake,[],sc_data_lake/data,https://discover-prod-datalake-v1.s3-us-west-2...,NaN,GSE137758_GPL18573,[Medulloblastoma],[None],"[PAX6, OTX2, MTOR, PAX3]",NaN,0,[None],discover-prod-datalake-v1,[None],GEO,Single cell,Tumorspheres were generated from 3 Group 3 MB ...,True,us-west-2
4,[Rattus norvegicus],15678.0,polly:data://sc_data_lake/data/GSE137763_GPL20...,4.0,NaN,Adult nucleus accumbens single-nucleus RNA-seq...,[None],NaN,NaN,sc-RNASeq,1659484621508,h5ad,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,[],sc_data_lake/data/GSE137763_GPL20084/GSE137763...,This dataset contains single-nucleus RNA seque...,sc_data_lake,[],sc_data_lake/data,https://discover-prod-datalake-v1.s3-us-west-2...,NaN,GSE137763_GPL20084,[Normal],"[cocaine, dopamine]",[None],NaN,0,[Sprague-Dawley],discover-prod-datalake-v1,"[nucleus accumbens, brain]",GEO,Single cell,This experiment contains 4 biological samples....,True,us-west-2


---

### Sample level querying to fetch sample level metadata in the schema

In [39]:
cells = library_client.query_metadata(
    f"""
    SELECT *
    FROM sc_data_lake.samples_singlecell
    WHERE src_dataset_id = 'GSE141578_GPL24676'
    """
)

Query execution succeeded (time taken: 12.01 seconds, data scanned: 129.195 MB)
Fetched 25294 rows


In [85]:
cells.head()

,gene_counts,organ,curated_raw_cell_type,gender,src_uri,sample_id,curated_gene_modified,curated_cell_line,title,src_dataset_id,timestamp_,percent_mito,donor,data_id,characteristics_ch1,curated_cell_type,clusters,curated_cell_ontology_id,strain,src_repo,id_key,organism_part,dataset_id,developmental_stage,curated_disease,curated_drug,curated_cell_type_singler,curated_gene,sample,version,umi_counts,curated_tissue,curated_marker_absent,name,curated_genetic_modification_type,location,is_current,age
0,1042,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCAAACTGTAGCTAAA,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.045862,NaN,gsm4208766_gcaaactgtagctaaa,tissue: cerebrospinal fluid|||cell type: immun...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,2900.0,[cerebrospinal fluid],[],GSM4208766:GCAAACTGTAGCTAAA,[],NaN,True,NaN
1,242,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCAAACTGTCTCACCT,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.065268,NaN,gsm4208766_gcaaactgtctcacct,tissue: cerebrospinal fluid|||cell type: immun...,NaN,14,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,429.0,[cerebrospinal fluid],[],GSM4208766:GCAAACTGTCTCACCT,[],NaN,True,NaN
2,870,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCAAACTTCATTGCGA,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.033478,NaN,gsm4208766_gcaaacttcattgcga,tissue: cerebrospinal fluid|||cell type: immun...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,2300.0,[cerebrospinal fluid],[],GSM4208766:GCAAACTTCATTGCGA,[],NaN,True,NaN
3,993,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCAATCATCGGAAATA,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.083566,NaN,gsm4208766_gcaatcatcggaaata,tissue: cerebrospinal fluid|||cell type: immun...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,2860.0,[cerebrospinal fluid],[],GSM4208766:GCAATCATCGGAAATA,[],NaN,True,NaN
4,1265,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCACATACAATCCGAT,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.067219,NaN,gsm4208766_gcacatacaatccgat,tissue: cerebrospinal fluid|||cell type: immun...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,5192.0,[cerebrospinal fluid],[],GSM4208766:GCACATACAATCCGAT,[],NaN,True,NaN


In [41]:
cells['title'].value_counts()

CSF__HC12    1997
CSF30_PD     1908
CSF__HC11    1861
CSF__HC10    1545
CSF__HC15    1544
CSF29_PD     1537
CSF14_PD     1489
CSF__HC16    1405
CSF__HC9     1340
CSF7_PD      1285
CSF__HC17    1268
CSF21_PD     1175
CSF24_PD     1080
CSF31_DLB    1047
CSF25_DLB     982
CSF__HC14     962
CSF28_HC      826
CSF27_PD      772
CSF10_PD      538
CSF__HC13     276
CSF32         261
CSF26_PD      196
Name: title, dtype: int64

In [48]:
cells['characteristics_ch1'].value_counts()

tissue: cerebrospinal fluid|||cell type: immune cells|||diagnosis: healthy control              13285
tissue: cerebrospinal fluid|||cell type: immune cells|||diagnosis: Parkinson's Disease           9980
tissue: cerebrospinal fluid|||cell type: immune cells|||diagnosis: Dementia with Lewy Bodies     2029
Name: characteristics_ch1, dtype: int64

In [49]:
import pandas as pd
pd.crosstab(cells['characteristics_ch1'],cells['title'])

title,CSF10_PD,CSF14_PD,CSF21_PD,CSF24_PD,CSF25_DLB,CSF26_PD,CSF27_PD,CSF28_HC,CSF29_PD,CSF30_PD,CSF31_DLB,CSF32,CSF7_PD,CSF__HC10,CSF__HC11,CSF__HC12,CSF__HC13,CSF__HC14,CSF__HC15,CSF__HC16,CSF__HC17,CSF__HC9
characteristics_ch1,,,,,,,,,,,,,,,,,,,,,,
tissue: cerebrospinal fluid|||cell type: immune cells|||diagnosis: Dementia with Lewy Bodies,0,0,0,0,982,0,0,0,0,0,1047,0,0,0,0,0,0,0,0,0,0,0
tissue: cerebrospinal fluid|||cell type: immune cells|||diagnosis: Parkinson's Disease,538,1489,1175,1080,0,196,772,0,1537,1908,0,0,1285,0,0,0,0,0,0,0,0,0
tissue: cerebrospinal fluid|||cell type: immune cells|||diagnosis: healthy control,0,0,0,0,0,0,0,826,0,0,0,261,0,1545,1861,1997,276,962,1544,1405,1268,1340


### Feature level querying to fetch feature level metadata in the schema

In [43]:
genes = library_client.query_metadata(
    f"""
    SELECT *
    FROM sc_data_lake.features_singlecell
    WHERE src_dataset_id = 'GSE141578_GPL24676'
    """
)

Query execution succeeded (time taken: 5.83 seconds, data scanned: 126.777 MB)
Fetched 17900 rows


In [86]:
genes.head()

,feature_id,src_repo,id_key,src_uri,dataset_id,n_cells,src_dataset_id,version,timestamp_,highly_variable,data_id,name,is_current
0,AL627309.1,sc_data_lake,kw_index,polly:data://sc_data_lake/data/GSE141578_GPL24...,NaN,27,GSE141578_GPL24676,0,1658313733838,False,al627309_1,AL627309.1,True
1,AL669831.5,sc_data_lake,kw_index,polly:data://sc_data_lake/data/GSE141578_GPL24...,NaN,216,GSE141578_GPL24676,0,1658313733838,False,al669831_5,AL669831.5,True
2,FAM87B,sc_data_lake,kw_index,polly:data://sc_data_lake/data/GSE141578_GPL24...,NaN,13,GSE141578_GPL24676,0,1658313733838,False,fam87b,FAM87B,True
3,LINC00115,sc_data_lake,kw_index,polly:data://sc_data_lake/data/GSE141578_GPL24...,NaN,270,GSE141578_GPL24676,0,1658313733838,False,linc00115,LINC00115,True
4,FAM41C,sc_data_lake,kw_index,polly:data://sc_data_lake/data/GSE141578_GPL24...,NaN,5,GSE141578_GPL24676,0,1658313733838,False,fam41c,FAM41C,True


In [87]:
#Querying the entire data matrix for this dataset ID
df = library_client.query_metadata(
    "SELECT * FROM sc_data_lake.datasets WHERE dataset_id = 'GSE141578_GPL24676'"
)
data_table_name = df['data_table_name'][0]

matrix_df = library_client.query_metadata(
    f"""
    SELECT *
        FROM data_matrices.{data_table_name}
    """
)
matrix_df.head()

Query execution succeeded (time taken: 1.24 seconds, data scanned: 0.378 MB)
Fetched 1 rows
Query execution succeeded (time taken: 8.72 seconds, data scanned: 0.429 MB)
Fetched 25294 rows


,row_id,acap3,megf6,tnfrsf9,kif1b,tnfrsf1b,ddi2,padi2,ubr4,ece1,zbtb40,c1qa,c1qc,c1qb,id3,runx3,rsrp1,stmn1,ubxn11,cd52,znf683,sfpq,ago3,macf1,plk3,tut4,usp24,jun,patj,pde4b,il12rb2,ac103591_3,ifi44l,gbp5,lrrc8b,tgfbr3,gfi1,ptbp2,slc35a3,dbt,cdc14a,al365361_1,chi3l2,c1orf162,slc16a1_as1,lrig2,tent5c,ac245014_3,cd160,txnip,fcgr1a,hist2h2bf,hist2h2aa4,ctss,gabpb2,s100a10,s100a11,s100a9,s100a8,s100a6,rps27,atp8b2,ash1l,lmna,fcrl3,cd1c,cd1e,mnda,pyhin1,fcer1a,fcrl6,cd84,slamf1,slamf7,fcer1g,fcgr2a,fcgr3a,fcgr2b,sh2d1b,pou2f1,xcl2,xcl1,f5,sell,faslg,zbtb37,cep350,glul,rgs16,c1orf21,fam129a,rgs1,rgs2,cfh,gpr25,kif21b,ipo9,ppp1r12b,btg2,atp2b4,lax1,elk4,slc41a1,fcmr,cd55,mir29b2chg,tmem63a,znf678,obscn,lyst,edaradd,akt3,ahctf1,linc01871,id2,linc00954,rhob,atad2b,selenoi,birc6,hnrnpll,eml4,zfp36l2,sptbn1,bcl11a,usp34,wdpcp,peli1,plek,aak1,tmsb10,capg,gnly,rnf103,cd8a,cd8b,linc01943,cytor,rgpd2,igkc,mal,linc00342,ankrd36c,dusp2,kansl3,cnnm3,ankrd36,ac092683_1,ankrd36b,il1r2,il18rap,ac012360_3,mir4435_2hg,bcl2l11,il1b,marco,clasp1,uggt1,mzt2b,mgat5,ccnt2,cxcr4,zeb2,nr4a2,stk39,mettl8,pdk1,cdca7,gpr155,chn1,ttn,ccdc141,linc01934,itga4,itgav,slc40a1,mfsd6,nemp2,stat1,nabp1,ankrd44,sf3b1,tyw5,casp8,cd28,ctla4,icos,ino80d,pikfyve,ikzf2,fn1,slc11a1,usp37,cyp27a1,epha4,itm2c,hdac4,pask,d2hgdh,pdcd1,thumpd3_as1,rpl32,slc6a6,nr2c2,ankrd28,slc4a7,eomes,cmc1,ccr4,epm2aip1,cx3cr1,nktr,snrk,fyco1,cxcr6,ccr5,nbeal2,znf589,rbm5,cish,rad54l2,dnah1,stab1,flnb,lrig1,foxp1,mtrnr2l12,cldnd1,cpox,zbtb11,cep97,nfkbiz,cblb,cd96,ccdc191,ac093010_2,tigit,zbtb20,golgb1,cd86,csta,parp15,parp14,ccdc14,zxdc,h1fx,nphp3,xrn1,serp1,dhx36,slc33a1,tiparp,ccnl1,phc3,pik3ca,ttc14,il1rap,ccdc50,tfrc,dlg1,uvssa,nsd2,mxd4,zfyve28,lyar,bod1l1,fgfbp2,arap2,dthd1,atp8a1,txk,fryl,hopx,spink2,cenpc,jchain,areg,cnot6l,lin54,plac8,spp1,fam13a,gprin3,lef1,tnip3,kiaa1109,ac097376_2,inpp4b,znf827,arhgap10,nr3c2,lrba,fbxw7,map9,hpgd,il7r,cplane1,rictor,selenop,emb,parp8,itga1,gzmk,gzma,il6st,pde4d,trim23,srek1,mast4,pik3r1,enc1,iqgap2,f2r,jmy,mtx3,serinc5,vcan,tmem161b,mef2c,slf1,pam,man2a1,camk4,march3,c5orf63,irf1,tcf7,tgfbi,egr1,mzb1,cd14,ndfip1,arhgap26,jakmip2,csf1r,cd74,galnt10,itk,adam19,pank3,dusp1,crebrf,nsd1,irf4,f13a1,ly86,adtrp,cd83,atxn1,e2f3,sox4,ripor2,hist1h4c,hist1h1e,hist1h1d,btn3a1,hist1h2bj,hist1h2ag,hist1h2bn,hla_g,ier3,hla_c,hla_b,tnf,ltb,lst1,ncr3,aif1,msh5,hspa1b,hla_dra,hla_drb5,hla_drb1,hla_dqa1,hla_dqb1,hla_dmb,hla_dma,hla_dpa1,hla_dpb1,rps18,lemd2,uhrf1bp1,cdkn1a,pim1,btbd9,trem2,cul9,aars2,runx2,ac019205_1,znf292,pnrc1,manea_dt,mms22l,prdm1,crybg1,fyn,mcm9,themis,samd3,med23,rps12,sgk1,al357060_1,tnfaip3,cited2,utrn,shprh,stxbp5,zc3h12d,syne1,arid1b,tmem181,sytl3,igf2r,rnaset2,ccr6,ints1,foxk1,actb,fscn1,glcci1,scin,snx13,cpvl,avl9,bbs9,aoah,trgc2,trgc1,trgv10,trgv9,trgv8,trgv7,trgv5,trgv3,trgv2,cdk13,stk17a,igfbp3,znf736,kctd7,tyw1b,trim73,fgl2,abcb1,akap9,casd1,lmtk2,trrap,pilrb,pilra,gigyf1,trim56,polr2j3_1,tmem168,ac016831_7,ac016831_1,ac016831_5,tmem140,ubn2,braf,kiaa1147,trbv2,trbv3_1,trbv4_1,trbv5_1,trbv6_1,trbv4_2,trbv6_2,trbv7_2,trbv7_3,trbv9,trbv10_2,trbv11_2,trbv6_5,trbv5_4,trbv6_6,trbv5_5,trbv7_6,trbv5_6,trbv7_9,trbv13,trbv10_3,trbv11_3,trbv12_3,trbv12_4,trbv14,trbv15,trbv18,trbv19,trbv20_1,trbv21_1,trbv24_1,trbv25_1,trbv27,trbv28,trbv29_1,trbc1,trbv30,ephb6,znf783,actr3c,tmem176b,kmt2c,insig1,il3ra,p2ry8,tmsb4x,sat1,cybb,rpgr,cask,kdm6a,timp1,foxp3,tspyl2,kantr,huwe1,vsig4,ogt,cxcr3,xist,jpx,ftx,gpr174,itm2a,brwd3,gprasp1,tsc22d3,rpl39,sh2d1a,cd40lg,flna,plxna3,ctsb,entpd4,nefl,clu,dusp4,saraf,kat6a,ap3m2,cebpd,tox,mybl1,ncoa2,msc,pag1,tp53inp1,vps13b,rnf19a,trps1,sntb1,atad2,tmem65,linc00861,pcat1,myc,sla,zfat,ago2,plec,mroh1,rfx3,ermp1,uhrf2,zdhhc21,ttc39b,haus6,aqp3,il11ra,pigo,rmrp,tpm2,zbtb5,cemip2,vps13a,cep78,ctsl,secisbp2,sema4d,mfsd14c,ugcg,akna,gsn,ttc16,fam102a,sptan1,prrc2b,rapgef1,tsc1,fcn1,notch1,ptgds,paxx,clic3,abca2,pgghg,ifitm2,ifitm1,ifitm3,ano9,irf7,dnhd1,trim66,mtrnr2l8,lyve1,pde3b,nucb2,prr5l,cry2,spi1,celf1,mpeg1,ms4a6a,

### Merging the datamatrix matrix with sample level metadata for a group of samples belonging to patients diagnosed with parkinson's disease

In [88]:
df = library_client.query_metadata(
    "SELECT * FROM sc_data_lake.datasets WHERE dataset_id = 'GSE141578_GPL24676'"
)
data_table_name = df['data_table_name'][0]

matrix_df = library_client.query_metadata(
    f"""
    SELECT *
    FROM (
        SELECT *
        FROM sc_data_lake.samples_singlecell
        WHERE src_dataset_id = 'GSE141578_GPL24676'
        AND title LIKE '%PD%'
    ) AS obs
    INNER JOIN (
        SELECT *
        FROM data_matrices.{data_table_name}
    ) AS matrix
    ON obs.name = matrix.obs_name
    """
)
matrix_df.head()

Query execution succeeded (time taken: 1.16 seconds, data scanned: 0.378 MB)
Fetched 1 rows
Query execution succeeded (time taken: 13.07 seconds, data scanned: 129.624 MB)
Fetched 9980 rows


,gene_counts,organ,curated_raw_cell_type,gender,src_uri,sample_id,curated_gene_modified,curated_cell_line,title,src_dataset_id,timestamp_,percent_mito,donor,data_id,characteristics_ch1,curated_cell_type,clusters,curated_cell_ontology_id,strain,src_repo,id_key,organism_part,dataset_id,developmental_stage,curated_disease,curated_drug,curated_cell_type_singler,curated_gene,sample,version,umi_counts,curated_tissue,curated_marker_absent,name,curated_genetic_modification_type,location,is_current,age,row_id,acap3,megf6,tnfrsf9,kif1b,tnfrsf1b,ddi2,padi2,ubr4,ece1,zbtb40,c1qa,c1qc,c1qb,id3,runx3,rsrp1,stmn1,ubxn11,cd52,znf683,sfpq,ago3,macf1,plk3,tut4,usp24,jun,patj,pde4b,il12rb2,ac103591_3,ifi44l,gbp5,lrrc8b,tgfbr3,gfi1,ptbp2,slc35a3,dbt,cdc14a,al365361_1,chi3l2,c1orf162,slc16a1_as1,lrig2,tent5c,ac245014_3,cd160,txnip,fcgr1a,hist2h2bf,hist2h2aa4,ctss,gabpb2,s100a10,s100a11,s100a9,s100a8,s100a6,rps27,atp8b2,ash1l,lmna,fcrl3,cd1c,cd1e,mnda,pyhin1,fcer1a,fcrl6,cd84,slamf1,slamf7,fcer1g,fcgr2a,fcgr3a,fcgr2b,sh2d1b,pou2f1,xcl2,xcl1,f5,sell,faslg,zbtb37,cep350,glul,rgs16,c1orf21,fam129a,rgs1,rgs2,cfh,gpr25,kif21b,ipo9,ppp1r12b,btg2,atp2b4,lax1,elk4,slc41a1,fcmr,cd55,mir29b2chg,tmem63a,znf678,obscn,lyst,edaradd,akt3,ahctf1,linc01871,id2,linc00954,rhob,atad2b,selenoi,birc6,hnrnpll,eml4,zfp36l2,sptbn1,bcl11a,usp34,wdpcp,peli1,plek,aak1,tmsb10,capg,gnly,rnf103,cd8a,cd8b,linc01943,cytor,rgpd2,igkc,mal,linc00342,ankrd36c,dusp2,kansl3,cnnm3,ankrd36,ac092683_1,ankrd36b,il1r2,il18rap,ac012360_3,mir4435_2hg,bcl2l11,il1b,marco,clasp1,uggt1,mzt2b,mgat5,ccnt2,cxcr4,zeb2,nr4a2,stk39,mettl8,pdk1,cdca7,gpr155,chn1,ttn,ccdc141,linc01934,itga4,itgav,slc40a1,mfsd6,nemp2,stat1,nabp1,ankrd44,sf3b1,tyw5,casp8,cd28,ctla4,icos,ino80d,pikfyve,ikzf2,fn1,slc11a1,usp37,cyp27a1,epha4,itm2c,hdac4,pask,d2hgdh,pdcd1,thumpd3_as1,rpl32,slc6a6,nr2c2,ankrd28,slc4a7,eomes,cmc1,ccr4,epm2aip1,cx3cr1,nktr,snrk,fyco1,cxcr6,ccr5,nbeal2,znf589,rbm5,cish,rad54l2,dnah1,stab1,flnb,lrig1,foxp1,mtrnr2l12,cldnd1,cpox,zbtb11,cep97,nfkbiz,cblb,cd96,ccdc191,ac093010_2,tigit,zbtb20,golgb1,cd86,csta,parp15,parp14,ccdc14,zxdc,h1fx,nphp3,xrn1,serp1,dhx36,slc33a1,tiparp,ccnl1,phc3,pik3ca,ttc14,il1rap,ccdc50,tfrc,dlg1,uvssa,nsd2,mxd4,zfyve28,lyar,bod1l1,fgfbp2,arap2,dthd1,atp8a1,txk,fryl,hopx,spink2,cenpc,jchain,areg,cnot6l,lin54,plac8,spp1,fam13a,gprin3,lef1,tnip3,kiaa1109,ac097376_2,inpp4b,znf827,arhgap10,nr3c2,lrba,fbxw7,map9,hpgd,il7r,cplane1,rictor,selenop,emb,parp8,itga1,gzmk,gzma,il6st,pde4d,trim23,srek1,mast4,pik3r1,enc1,iqgap2,f2r,jmy,mtx3,serinc5,vcan,tmem161b,mef2c,slf1,pam,man2a1,camk4,march3,c5orf63,irf1,tcf7,tgfbi,egr1,mzb1,cd14,ndfip1,arhgap26,jakmip2,csf1r,cd74,galnt10,itk,adam19,pank3,dusp1,crebrf,nsd1,irf4,f13a1,ly86,adtrp,cd83,atxn1,e2f3,sox4,ripor2,hist1h4c,hist1h1e,hist1h1d,btn3a1,hist1h2bj,hist1h2ag,hist1h2bn,hla_g,ier3,hla_c,hla_b,tnf,ltb,lst1,ncr3,aif1,msh5,hspa1b,hla_dra,hla_drb5,hla_drb1,hla_dqa1,hla_dqb1,hla_dmb,hla_dma,hla_dpa1,hla_dpb1,rps18,lemd2,uhrf1bp1,cdkn1a,pim1,btbd9,trem2,cul9,aars2,runx2,ac019205_1,znf292,pnrc1,manea_dt,mms22l,prdm1,crybg1,fyn,mcm9,themis,samd3,med23,rps12,sgk1,al357060_1,tnfaip3,cited2,utrn,shprh,stxbp5,zc3h12d,syne1,arid1b,tmem181,sytl3,igf2r,rnaset2,ccr6,ints1,foxk1,actb,fscn1,glcci1,scin,snx13,cpvl,avl9,bbs9,aoah,trgc2,trgc1,trgv10,trgv9,trgv8,trgv7,trgv5,trgv3,trgv2,cdk13,stk17a,igfbp3,znf736,kctd7,tyw1b,trim73,fgl2,abcb1,akap9,casd1,lmtk2,trrap,pilrb,pilra,gigyf1,trim56,polr2j3_1,tmem168,ac016831_7,ac016831_1,ac016831_5,tmem140,ubn2,braf,kiaa1147,trbv2,trbv3_1,trbv4_1,trbv5_1,trbv6_1,trbv4_2,trbv6_2,trbv7_2,trbv7_3,trbv9,trbv10_2,trbv11_2,trbv6_5,trbv5_4,trbv6_6,trbv5_5,trbv7_6,trbv5_6,trbv7_9,trbv13,trbv10_3,trbv11_3,trbv12_3,trbv12_4,trbv14,trbv15,trbv18,trbv19,trbv20_1,trbv21_1,trbv24_1,trbv25_1,trbv27,trbv28,trbv29_1,trbc1,trbv30,ephb6,znf783,actr3c,tmem176b,kmt2c,insig1,il3ra,p2ry8,tmsb4x,sat1,cybb,rpgr,cask,kdm6a,timp1,foxp3,tspyl2,kantr,huwe1,vsig4,ogt,cxcr3,xist,jpx,ftx,gpr174,itm2a,brwd3,gprasp1,tsc22d3,rpl39,sh2d1a,cd40lg,flna,

### Complex querying for +ve expression of cd63 gene in samples belonging to Parkinson disease

In [89]:
## cells where expression of cd63 is > 0
df = library_client.query_metadata(
    "SELECT * FROM sc_data_lake.datasets WHERE dataset_id = 'GSE141578_GPL24676'"
)
data_table_name = df['data_table_name'][0]

matrix_df = library_client.query_metadata(
    f"""
    SELECT *
    FROM (
        SELECT *
        FROM sc_data_lake.samples_singlecell
        WHERE src_dataset_id = 'GSE141578_GPL24676'
        AND  title LIKE '%PD%'
    ) AS obs
    INNER JOIN (
        SELECT *
        FROM data_matrices.{data_table_name}
        WHERE cd63 > 0
    ) AS matrix
    ON obs.name = matrix.obs_name
    """
)
matrix_df.head()

Query execution succeeded (time taken: 1.38 seconds, data scanned: 0.378 MB)
Fetched 1 rows
Query execution succeeded (time taken: 9.55 seconds, data scanned: 129.624 MB)
Fetched 7 rows


,gene_counts,organ,curated_raw_cell_type,gender,src_uri,sample_id,curated_gene_modified,curated_cell_line,title,src_dataset_id,timestamp_,percent_mito,donor,data_id,characteristics_ch1,curated_cell_type,clusters,curated_cell_ontology_id,strain,src_repo,id_key,organism_part,dataset_id,developmental_stage,curated_disease,curated_drug,curated_cell_type_singler,curated_gene,sample,version,umi_counts,curated_tissue,curated_marker_absent,name,curated_genetic_modification_type,location,is_current,age,row_id,acap3,megf6,tnfrsf9,kif1b,tnfrsf1b,ddi2,padi2,ubr4,ece1,zbtb40,c1qa,c1qc,c1qb,id3,runx3,rsrp1,stmn1,ubxn11,cd52,znf683,sfpq,ago3,macf1,plk3,tut4,usp24,jun,patj,pde4b,il12rb2,ac103591_3,ifi44l,gbp5,lrrc8b,tgfbr3,gfi1,ptbp2,slc35a3,dbt,cdc14a,al365361_1,chi3l2,c1orf162,slc16a1_as1,lrig2,tent5c,ac245014_3,cd160,txnip,fcgr1a,hist2h2bf,hist2h2aa4,ctss,gabpb2,s100a10,s100a11,s100a9,s100a8,s100a6,rps27,atp8b2,ash1l,lmna,fcrl3,cd1c,cd1e,mnda,pyhin1,fcer1a,fcrl6,cd84,slamf1,slamf7,fcer1g,fcgr2a,fcgr3a,fcgr2b,sh2d1b,pou2f1,xcl2,xcl1,f5,sell,faslg,zbtb37,cep350,glul,rgs16,c1orf21,fam129a,rgs1,rgs2,cfh,gpr25,kif21b,ipo9,ppp1r12b,btg2,atp2b4,lax1,elk4,slc41a1,fcmr,cd55,mir29b2chg,tmem63a,znf678,obscn,lyst,edaradd,akt3,ahctf1,linc01871,id2,linc00954,rhob,atad2b,selenoi,birc6,hnrnpll,eml4,zfp36l2,sptbn1,bcl11a,usp34,wdpcp,peli1,plek,aak1,tmsb10,capg,gnly,rnf103,cd8a,cd8b,linc01943,cytor,rgpd2,igkc,mal,linc00342,ankrd36c,dusp2,kansl3,cnnm3,ankrd36,ac092683_1,ankrd36b,il1r2,il18rap,ac012360_3,mir4435_2hg,bcl2l11,il1b,marco,clasp1,uggt1,mzt2b,mgat5,ccnt2,cxcr4,zeb2,nr4a2,stk39,mettl8,pdk1,cdca7,gpr155,chn1,ttn,ccdc141,linc01934,itga4,itgav,slc40a1,mfsd6,nemp2,stat1,nabp1,ankrd44,sf3b1,tyw5,casp8,cd28,ctla4,icos,ino80d,pikfyve,ikzf2,fn1,slc11a1,usp37,cyp27a1,epha4,itm2c,hdac4,pask,d2hgdh,pdcd1,thumpd3_as1,rpl32,slc6a6,nr2c2,ankrd28,slc4a7,eomes,cmc1,ccr4,epm2aip1,cx3cr1,nktr,snrk,fyco1,cxcr6,ccr5,nbeal2,znf589,rbm5,cish,rad54l2,dnah1,stab1,flnb,lrig1,foxp1,mtrnr2l12,cldnd1,cpox,zbtb11,cep97,nfkbiz,cblb,cd96,ccdc191,ac093010_2,tigit,zbtb20,golgb1,cd86,csta,parp15,parp14,ccdc14,zxdc,h1fx,nphp3,xrn1,serp1,dhx36,slc33a1,tiparp,ccnl1,phc3,pik3ca,ttc14,il1rap,ccdc50,tfrc,dlg1,uvssa,nsd2,mxd4,zfyve28,lyar,bod1l1,fgfbp2,arap2,dthd1,atp8a1,txk,fryl,hopx,spink2,cenpc,jchain,areg,cnot6l,lin54,plac8,spp1,fam13a,gprin3,lef1,tnip3,kiaa1109,ac097376_2,inpp4b,znf827,arhgap10,nr3c2,lrba,fbxw7,map9,hpgd,il7r,cplane1,rictor,selenop,emb,parp8,itga1,gzmk,gzma,il6st,pde4d,trim23,srek1,mast4,pik3r1,enc1,iqgap2,f2r,jmy,mtx3,serinc5,vcan,tmem161b,mef2c,slf1,pam,man2a1,camk4,march3,c5orf63,irf1,tcf7,tgfbi,egr1,mzb1,cd14,ndfip1,arhgap26,jakmip2,csf1r,cd74,galnt10,itk,adam19,pank3,dusp1,crebrf,nsd1,irf4,f13a1,ly86,adtrp,cd83,atxn1,e2f3,sox4,ripor2,hist1h4c,hist1h1e,hist1h1d,btn3a1,hist1h2bj,hist1h2ag,hist1h2bn,hla_g,ier3,hla_c,hla_b,tnf,ltb,lst1,ncr3,aif1,msh5,hspa1b,hla_dra,hla_drb5,hla_drb1,hla_dqa1,hla_dqb1,hla_dmb,hla_dma,hla_dpa1,hla_dpb1,rps18,lemd2,uhrf1bp1,cdkn1a,pim1,btbd9,trem2,cul9,aars2,runx2,ac019205_1,znf292,pnrc1,manea_dt,mms22l,prdm1,crybg1,fyn,mcm9,themis,samd3,med23,rps12,sgk1,al357060_1,tnfaip3,cited2,utrn,shprh,stxbp5,zc3h12d,syne1,arid1b,tmem181,sytl3,igf2r,rnaset2,ccr6,ints1,foxk1,actb,fscn1,glcci1,scin,snx13,cpvl,avl9,bbs9,aoah,trgc2,trgc1,trgv10,trgv9,trgv8,trgv7,trgv5,trgv3,trgv2,cdk13,stk17a,igfbp3,znf736,kctd7,tyw1b,trim73,fgl2,abcb1,akap9,casd1,lmtk2,trrap,pilrb,pilra,gigyf1,trim56,polr2j3_1,tmem168,ac016831_7,ac016831_1,ac016831_5,tmem140,ubn2,braf,kiaa1147,trbv2,trbv3_1,trbv4_1,trbv5_1,trbv6_1,trbv4_2,trbv6_2,trbv7_2,trbv7_3,trbv9,trbv10_2,trbv11_2,trbv6_5,trbv5_4,trbv6_6,trbv5_5,trbv7_6,trbv5_6,trbv7_9,trbv13,trbv10_3,trbv11_3,trbv12_3,trbv12_4,trbv14,trbv15,trbv18,trbv19,trbv20_1,trbv21_1,trbv24_1,trbv25_1,trbv27,trbv28,trbv29_1,trbc1,trbv30,ephb6,znf783,actr3c,tmem176b,kmt2c,insig1,il3ra,p2ry8,tmsb4x,sat1,cybb,rpgr,cask,kdm6a,timp1,foxp3,tspyl2,kantr,huwe1,vsig4,ogt,cxcr3,xist,jpx,ftx,gpr174,itm2a,brwd3,gprasp1,tsc22d3,rpl39,sh2d1a,cd40lg,flna,

### Identifing other datasets in the Atlas which also has samples of patients suffering from Parkinson's disease

In [99]:
query = """SELECT dataset_id, total_num_samples, description, data_table_name FROM sc_data_lake.datasets WHERE CONTAINS(curated_disease, 'Parkinson Disease')"""
result = library_client.query_metadata(query)
result

Query execution succeeded (time taken: 1.33 seconds, data scanned: 0.231 MB)
Fetched 23 rows


,dataset_id,total_num_samples,description,data_table_name
0,E-MTAB-7303,119,Single cell sequencing of iPSC-dopamine neuron...,NaN
1,E-MTAB-9154,6,Single cell transcriptomics of iPSC derived ne...,NaN
2,GSE108020_GPL17021,429,Single-cell RNA-seq of mouse dopaminergic neur...,sc_data_lake__gse108020_gpl17021
3,GSE115070_GPL17021,280,Studying the genetic heterogeneity in mouse do...,sc_data_lake__gse115070_gpl17021
4,GSE116138_GPL13112,1414,Single-cell RNA Sequencing Reveals Midbrain Do...,sc_data_lake__gse116138_gpl13112
5,GSE117394_GPL13112,56,Molecular analysis of the midbrain dopaminergi...,sc_data_lake__gse117394_gpl13112
6,GSE118412_GPL16791,1396,Single Cell Transcriptomics Identifies Stem Ce...,NaN
7,GSE128040_GPL18573,2,Neural stem cells of Parkinson’s disease patie...,sc_data_lake__gse128040_gpl18573
8,GSE132758_GPL18573,8,Single cell transcriptomics identifies stem ce...,NaN
9,GSE133894_GPL18573,4,The Parkinson's disease associated mutation LR...,sc_data_lake__gse133894_gpl18573


### UNION queries
First we'll explore the metadata of two datasets and then perform UNION for them

In [90]:
query = """SELECT * FROM sc_data_lake.samples_singlecell WHERE src_dataset_id = 'GSE184950_GPL24676'"""
result = library_client.query_metadata(query)
result.head()

Query execution succeeded (time taken: 18.60 seconds, data scanned: 4162.679 MB)
Fetched 260413 rows


,gene_counts,organ,curated_raw_cell_type,gender,src_uri,sample_id,curated_gene_modified,curated_cell_line,title,src_dataset_id,timestamp_,percent_mito,donor,data_id,characteristics_ch1,curated_cell_type,clusters,curated_cell_ontology_id,strain,src_repo,id_key,organism_part,dataset_id,developmental_stage,curated_disease,curated_drug,curated_cell_type_singler,curated_gene,sample,version,umi_counts,curated_tissue,curated_marker_absent,name,curated_genetic_modification_type,location,is_current,age
0,232,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602321:CTCTCAGTCGACCTAA,[none],[none],A38,GSE184950_GPL24676,1651066851721,0.007452,NaN,gsm5602321_ctctcagtcgacctaa,tissue: Substanstia nigra|||disease state: Par...,NaN,30,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602321,0,671.0,[brain],[],GSM5602321:CTCTCAGTCGACCTAA,[],NaN,True,NaN
1,466,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602338:TTGCGTCGTCTCCTGT,[none],[none],D27,GSE184950_GPL24676,1651066853837,0.102649,NaN,gsm5602338_ttgcgtcgtctcctgt,tissue: Substanstia nigra|||disease state: Par...,NaN,4,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602338,0,604.0,[brain],[],GSM5602338:TTGCGTCGTCTCCTGT,[],NaN,True,NaN
2,1390,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602320:ATTCATCGTAGAGGAA,[none],[none],A31,GSE184950_GPL24676,1651066851547,0.183482,NaN,gsm5602320_attcatcgtagaggaa,tissue: Substanstia nigra|||disease state: Par...,NaN,22,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602320,0,2349.0,[brain],[],GSM5602320:ATTCATCGTAGAGGAA,[],NaN,True,NaN
3,579,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602332:CCTAACCTCGGTCGAC,[none],[none],C20,GSE184950_GPL24676,1651066852568,0.098361,NaN,gsm5602332_cctaacctcggtcgac,tissue: Substanstia nigra|||disease state: Par...,NaN,9,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602332,0,793.0,[brain],[],GSM5602332:CCTAACCTCGGTCGAC,[],NaN,True,NaN
4,855,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602337:ATTACCTAGCCTCCAG,[none],[none],D20,GSE184950_GPL24676,1651066853352,0.064493,NaN,gsm5602337_attacctagcctccag,tissue: Substanstia nigra|||disease state: Par...,NaN,7,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602337,0,1380.0,[brain],[],GSM5602337:ATTACCTAGCCTCCAG,[],NaN,True,NaN


In [78]:
result['title'].value_counts()

A29    21481
C20    18642
D27    17382
p5     16084
D20    15117
D16    14924
B19    12819
A18    12597
p4     12442
C26    12423
A38    11642
A31    11556
A2     10439
C8      9907
C11     9602
A15     8710
C1      7196
p12     6172
C18     5587
D33     4339
B9      4065
D29     3286
D3      3284
B13     3042
B33     2667
C29     1396
B28     1136
B5       901
A10      675
B25      414
p9       360
D8        82
p7        44
Name: title, dtype: int64

In [79]:
result['characteristics_ch1'].value_counts()

tissue: Substanstia nigra|||disease state: Unaffected Control|||brain bank donor id: HBHF|||age: 93|||gender: Female|||race: White|||ethnicity: Not Hispanic or  Latino|||postmortem interval hours: 23.5|||braak stage: N/A                     28526
tissue: Substanstia nigra|||disease state: Parkinson's Disease Dementia|||brain bank donor id: HBOX|||age: 82|||gender: Male|||race: White|||ethnicity: Not Hispanic or  Latino|||postmortem interval hours: 4.2|||braak stage: Stage VI         21481
tissue: Substanstia nigra|||disease state: Parkinson's Disease|||brain bank donor id: HBFS|||age: 91|||gender: Male|||race: White|||ethnicity: Not Hispanic or  Latino|||postmortem interval hours: 16.06|||braak stage: N/A                     18642
tissue: Substanstia nigra|||disease state: Parkinson's Disease|||brain bank donor id: HAAB|||age: 73|||gender: Male|||race: White|||ethnicity: Not Hispanic or Latino|||postmortem interval hours: 12.25|||braak stage: Stage IV                 17382
tissue: Subs

In [91]:
query = """SELECT * FROM sc_data_lake.samples_singlecell WHERE src_dataset_id = 'GSE141578_GPL24676'"""
result_1 = library_client.query_metadata(query)
result_1.head()

Query execution succeeded (time taken: 10.95 seconds, data scanned: 129.195 MB)
Fetched 25294 rows


,gene_counts,organ,curated_raw_cell_type,gender,src_uri,sample_id,curated_gene_modified,curated_cell_line,title,src_dataset_id,timestamp_,percent_mito,donor,data_id,characteristics_ch1,curated_cell_type,clusters,curated_cell_ontology_id,strain,src_repo,id_key,organism_part,dataset_id,developmental_stage,curated_disease,curated_drug,curated_cell_type_singler,curated_gene,sample,version,umi_counts,curated_tissue,curated_marker_absent,name,curated_genetic_modification_type,location,is_current,age
0,1042,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCAAACTGTAGCTAAA,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.045862,NaN,gsm4208766_gcaaactgtagctaaa,tissue: cerebrospinal fluid|||cell type: immun...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,2900.0,[cerebrospinal fluid],[],GSM4208766:GCAAACTGTAGCTAAA,[],NaN,True,NaN
1,242,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCAAACTGTCTCACCT,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.065268,NaN,gsm4208766_gcaaactgtctcacct,tissue: cerebrospinal fluid|||cell type: immun...,NaN,14,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,429.0,[cerebrospinal fluid],[],GSM4208766:GCAAACTGTCTCACCT,[],NaN,True,NaN
2,870,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCAAACTTCATTGCGA,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.033478,NaN,gsm4208766_gcaaacttcattgcga,tissue: cerebrospinal fluid|||cell type: immun...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,2300.0,[cerebrospinal fluid],[],GSM4208766:GCAAACTTCATTGCGA,[],NaN,True,NaN
3,993,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCAATCATCGGAAATA,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.083566,NaN,gsm4208766_gcaatcatcggaaata,tissue: cerebrospinal fluid|||cell type: immun...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,2860.0,[cerebrospinal fluid],[],GSM4208766:GCAATCATCGGAAATA,[],NaN,True,NaN
4,1265,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE141578_GPL24...,GSM4208766:GCACATACAATCCGAT,[none],[none],CSF14_PD,GSE141578_GPL24676,1662445504288,0.067219,NaN,gsm4208766_gcacatacaatccgat,tissue: cerebrospinal fluid|||cell type: immun...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM4208766,0,5192.0,[cerebrospinal fluid],[],GSM4208766:GCACATACAATCCGAT,[],NaN,True,NaN


In [101]:
result_1['title'].value_counts()

CSF__HC12    1997
CSF30_PD     1908
CSF__HC11    1861
CSF__HC10    1545
CSF__HC15    1544
CSF29_PD     1537
CSF14_PD     1489
CSF__HC16    1405
CSF__HC9     1340
CSF7_PD      1285
CSF__HC17    1268
CSF21_PD     1175
CSF24_PD     1080
CSF31_DLB    1047
CSF25_DLB     982
CSF__HC14     962
CSF28_HC      826
CSF27_PD      772
CSF10_PD      538
CSF__HC13     276
CSF32         261
CSF26_PD      196
Name: title, dtype: int64

In [68]:
## For hypothetically selected cohorts from both datasets, performing a union query

In [100]:
query = """SELECT * FROM sc_data_lake.samples_singlecell WHERE src_dataset_id = 'GSE141578_GPL24676' AND title = '%PD%'
           UNION
           SELECT * FROM sc_data_lake.samples_singlecell WHERE src_dataset_id = 'GSE184950_GPL24676' AND title = 'C20'
           """
result_combined = library_client.query_metadata(query)
result_combined.head()

Query execution succeeded (time taken: 12.79 seconds, data scanned: 4256.772 MB)
Fetched 18642 rows


,gene_counts,organ,curated_raw_cell_type,gender,src_uri,sample_id,curated_gene_modified,curated_cell_line,title,src_dataset_id,timestamp_,percent_mito,donor,data_id,characteristics_ch1,curated_cell_type,clusters,curated_cell_ontology_id,strain,src_repo,id_key,organism_part,dataset_id,developmental_stage,curated_disease,curated_drug,curated_cell_type_singler,curated_gene,sample,version,umi_counts,curated_tissue,curated_marker_absent,name,curated_genetic_modification_type,location,is_current,age
0,956,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602332:GCATTAGCAGCGTGAA,[none],[none],C20,GSE184950_GPL24676,1651066852617,0.032192,NaN,gsm5602332_gcattagcagcgtgaa,tissue: Substanstia nigra|||disease state: Par...,NaN,11,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602332,0,1460.0,[brain],[],GSM5602332:GCATTAGCAGCGTGAA,[],NaN,True,NaN
1,899,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602332:CACGTGGCAATAGTGA,[none],[none],C20,GSE184950_GPL24676,1651066852547,0.093417,NaN,gsm5602332_cacgtggcaatagtga,tissue: Substanstia nigra|||disease state: Par...,NaN,11,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602332,0,1595.0,[brain],[],GSM5602332:CACGTGGCAATAGTGA,[],NaN,True,NaN
2,683,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602332:CCACTTGAGTCATGCT,[none],[none],C20,GSE184950_GPL24676,1651066852562,0.095960,NaN,gsm5602332_ccacttgagtcatgct,tissue: Substanstia nigra|||disease state: Par...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602332,0,990.0,[brain],[],GSM5602332:CCACTTGAGTCATGCT,[],NaN,True,NaN
3,1321,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602332:GCCGATGCAGCTATTG,[none],[none],C20,GSE184950_GPL24676,1651066852620,0.054389,NaN,gsm5602332_gccgatgcagctattg,tissue: Substanstia nigra|||disease state: Par...,NaN,11,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602332,0,2666.0,[brain],[],GSM5602332:GCCGATGCAGCTATTG,[],NaN,True,NaN
4,597,NaN,NaN,NaN,polly:data://sc_data_lake/data/GSE184950_GPL24...,GSM5602332:ATTCTTGCACGTCTCT,[none],[none],C20,GSE184950_GPL24676,1651066852539,0.161476,NaN,gsm5602332_attcttgcacgtctct,tissue: Substanstia nigra|||disease state: Par...,NaN,3,NaN,NaN,sc_data_lake,kw_column,NaN,[],NaN,[Parkinson Disease],[none],NaN,[],GSM5602332,0,867.0,[brain],[],GSM5602332:ATTCTTGCACGTCTCT,[],NaN,True,NaN
